<a href="https://colab.research.google.com/github/suraj1045/Gen-AI-projects/blob/main/Sentiment_analysis/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Utilizing a pre-trained transformer model from hugging face's transformer library to perform sentiment analysis. Evalute real-world review datasets and interpret the model's predictions.

In [1]:
import torch
torch.cuda.is_available()

False

In [ ]:
!pip install datasets transformers huggingface_hub
!apt-get install git-lfs
!pip install datasets
!pip install evaluate

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.7 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
imdb = load_dataset("imdb")

README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
# creating small datasets from imdb for faster training
strain_dataset = imdb["train"].shuffle(seed=42).select([i for i in list(range(3000))])
stest_dataset = imdb["test"].shuffle(seed=42).select([i for i in list(range(300))])
print(strain_dataset)
print(stest_dataset)

Dataset({
    features: ['text', 'label'],
    num_rows: 3000
})
Dataset({
    features: ['text', 'label'],
    num_rows: 300
})


In [ ]:
# preprocess data using distilBERT tokenizer
# preprocessing: the process of preparing the raw data to be used for a particular use case
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
# prepare the text inputs for the model for both splits of the dataset using map method

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenised_train = strain_dataset.map(preprocess_function,batched = True)
tokenised_test = stest_dataset.map(preprocess_function, batched = True)

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [ ]:
#using data_collector to speed up training by converting training samples into pytorch tensores and concatenating them with correct amount of padding
from transformers import DataCollatorWithPadding
data_collector = DataCollatorWithPadding(tokenizer = tokenizer)

In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

In [ ]:
#model evaluation
import numpy as np
import evaluate

def compute_metrics(eval_pred):
  load_accuracy = evaluate.load("accuracy")
  load_f1 = evaluate.load("f1")

  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
  f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
  return {"accuracy": accuracy, "f1": f1}

In [ ]:
#login
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
#defining training arguments and trainer

from transformers import TrainingArguments, Trainer
repo_name = "finetuning-sentiment-model-3000-samples"

training_args = TrainingArguments(
    output_dir=repo_name,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size = 16,
    num_train_epochs = 2,
    weight_decay = 0.01,
    save_strategy = "epoch",
    push_to_hub = True,
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenised_train,
    eval_dataset = tokenised_test,
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics,
)

NameError: name 'model' is not defined

In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()